## [부록 E] RAG를 활용한 스마트폰 추천받기

In [1]:
# 문서 데이터베이스
documents = [
    "Python은 프로그래밍 언어입니다.",
    "Python은 데이터 과학에 많이 사용됩니다.",
    "Python은 초보자도 배우기 쉬운 언어입니다."
]

# 간단한 키워드 기반 문서 검색 함수
#  - query: 사용자가 입력한 검색어
#  - documents: 검색 대상 문서 리스트
# return: 검색어를 포함한 문서 리스트
def retrieve_documents(query, documents):
    return [doc for doc in documents if query in doc]

# 검색된 문서를 기반으로 답변 생성 함수
#  - query: 사용자가 입력한 질문
#  - retrieved_docs: 검색된 문서 리스트
#  - return: 생성된 답변 (첫 번째 검색 결과를 사용)
def generate_answer(query, retrieved_docs):
    if not retrieved_docs:
        return "관련 정보를 찾을 수 없습니다."
    return retrieved_docs[0]

# RAG 시연
queries = [
    "Python",        # Python과 관련된 문서 검색
    "데이터 과학",   # '데이터 과학' 키워드를 포함한 문서 검색
    "프로그래밍"     # '프로그래밍' 키워드를 포함한 문서 검색
]

# 각 질문에 대해 검색 및 답변 생성 실행
for query in queries:
    print(f"질문: {query}")                                # 사용자 질문 출력
    retrieved_docs = retrieve_documents(query, documents)  # 검색 단계
    print(f"검색된 문서 : {retrieved_docs}")
    answer = generate_answer(query, retrieved_docs)        # 생성 단계
    print(f"답변: {answer}\n")                             # 생성된 답변 출력


질문: Python
검색된 문서 : ['Python은 프로그래밍 언어입니다.', 'Python은 데이터 과학에 많이 사용됩니다.', 'Python은 초보자도 배우기 쉬운 언어입니다.']
답변: Python은 프로그래밍 언어입니다.

질문: 데이터 과학
검색된 문서 : ['Python은 데이터 과학에 많이 사용됩니다.']
답변: Python은 데이터 과학에 많이 사용됩니다.

질문: 프로그래밍
검색된 문서 : ['Python은 프로그래밍 언어입니다.']
답변: Python은 프로그래밍 언어입니다.



### RAG 두번째 예제

In [2]:
!pip install openai
!pip install httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


In [5]:
from openai import OpenAI
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import os

# OpenAI API 초기화
with open("chatgpt.env") as env:
  for line in env:
    key, value = line.strip().split("=")
    os.environ[key] = value

client = OpenAI(api_key  = os.environ['API_KEY'])

# 문서 데이터셋 정의
documents = [
    "갤럭시북3 120만원 i5 16GB RAM 512GB SSD 별점4.5 2023년 출시 배송무료 리뷰 152개",
    "그램 16Z90R 135만원 i5 16GB RAM 256GB SSD 별점4.7 당일배송 리뷰 230개",
    "맥북 에어 M2 160만원 8GB RAM 256GB SSD 별점4.8 배송무료 리뷰 340개"
]

# 문서 임베딩 생성
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
document_embeddings = embedding_model.encode(documents)

# 사용자 질문 입력
user_query = "가성비 좋은 노트북 추천 부탁합니다."
query_embedding = embedding_model.encode([user_query])

# 문서 검색 (코사인 유사도 계산)
similarities = cosine_similarity(query_embedding, document_embeddings)
most_relevant_doc_index = np.argmax(similarities)
retrieved_document = documents[most_relevant_doc_index]

# ChatGPT를 사용한 답변 생성
prompt = f"문서: {retrieved_document}\n\n질문: {user_query}\n\n답변:"
response = client.chat.completions.create(
    model="gpt-4o",  # 최신 모델 사용
    messages=[{"role": "system", "content": "You are a helpful assistant."},
              {"role": "user", "content": prompt}],
    max_tokens=300,
    temperature=0.7
)

answer = response.choices[0].message.content.strip()
print("질문:", user_query)
print("유사도:", similarities)
print("검색된 문서:", retrieved_document)
print(f"[생성된 답변]: {answer}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

질문: 가성비 좋은 노트북 추천 부탁합니다.
유사도: [[0.26008344 0.21437721 0.2215741 ]]
검색된 문서: 갤럭시북3 120만원 i5 16GB RAM 512GB SSD 별점4.5 2023년 출시 배송무료 리뷰 152개
[생성된 답변]: 갤럭시북3을 고려해보시는 것도 좋을 것 같습니다. 이 모델은 2023년에 출시되었으며, 인텔 i5 프로세서, 16GB RAM, 512GB SSD를 갖추고 있어 일상적인 작업과 멀티태스킹에 충분한 성능을 제공합니다. 또한, 120만원이라는 가격대에서 이 정도의 사양과 무료 배송 혜택을 제공하는 점이 매력적입니다. 리뷰도 152개나 있으며, 평균 별점이 4.5로 높은 평가를 받고 있어 사용자 만족도가 높은 제품으로 보입니다. 이러한 점들을 고려했을 때, 갤럭시북3은 가성비 좋은 노트북으로 추천할 만합니다.


In [7]:
!pip install openai pypdf langchain langchain_core langchain_openai langchain_chroma langchain-community langchainhub

In [8]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

import os
from openai import OpenAI

def init_api():
    with open("chatgpt.env") as env:
       for line in env:
           key, value = line.strip().split("=")
           os.environ[key] = value

init_api()
os.environ["OPENAI_API_KEY"] = os.environ.get("API_KEY")

# PDF 파일이 저장된 폴더 경로를 설정
folder_path = './pdf_data/'

# 추출된 텍스트를 저장할 리스트
all_texts = []

# 텍스트를 분할할 때 사용할 CharacterTextSplitter 객체 생성
# separator = "\n", # 텍스트를 분할할 때 사용할 구분자
# chunk_size = 1000, # 각 분할된 텍스트의 최대 길이
# chunk_overlap = 50 # 분할된 텍스트 간의 중첩 길이
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap = 50)

# 지정된 폴더 내 모든 파일을 확인
for filename in os.listdir(folder_path):

    # 파일이 PDF 형식인 경우에만 처리
    if filename.endswith(".pdf"):

        # PDF 파일을 로드하여 raw_documents에 저장
        raw_documents = PyPDFLoader(folder_path + '/' + filename).load()

        # 로드된 문서를 분할하여 documents에 저장
        documents = text_splitter.split_documents(raw_documents)

        # 분할된 텍스트를 리스트에 추가
        all_texts.extend(documents)

# 분할된 텍스트를 임베딩으로 변환하고 Chroma 데이터베이스에 저장
db = Chroma.from_documents(all_texts, OpenAIEmbeddings())

# 데이터베이스에서 검색을 수행할 수 있는 retriever 객체 생성
# search_kwargs: 검색 시 반환할 결과 수를 설정
retriever = db.as_retriever(search_kwargs={"k": 10})

prompt_template = ChatPromptTemplate.from_template(
    "당신은 질문 답변 작업의 영리하고 창의적인 어시스턴트입니다. "
    "다음 문맥을 사용하여 질문에 답하세요. "
    "정확하고 신뢰성 있는 정보를 제공하고, 모르는 내용은 '모르겠습니다'라고 답변하세요. "
    "답변은 명확하고 간결하게, 최대 세 문장 이내로 작성하세요. 메타데이터나 추가적인 중요한 정보를 포함하도록 하세요. "
    "한국어로 작성합니다.\n\n"
    "질문: {question}\n"
    "문맥: {context}\n"
    "답변:"
)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_template
    | ChatOpenAI()
    | StrOutputParser()
)

def chat_with_user(user_message):
    ai_message = chain.invoke(user_message)
    return ai_message

# 대화 루프 시작
while True:
    message = input("USER :(quit or q : 종료)  ")
    if message.lower() == "quit" or message.lower() == "q":
        break

    ai_message = chat_with_user(message)
    print(f" AI : {ai_message}")

USER :(quit or q : 종료)  토픽 모델링이 뭐니?
 AI : 주제 모델링은 대량의 텍스트 데이터에서 숨겨진 주제 구조를 발견하는 통계적 모델링 기법입니다. 가장 널리 사용되는 방법으로 LDA(Latent Dirichlet Allocation)가 있습니다. 이를 통해 문서 집합에서 추상적인 '주제'를 자동으로 발견하고, 각 문서가 이러한 주제들의 혼합으로 구성되어 있다고 가정합니다.
USER :(quit or q : 종료)  빈도분석은 뭘까?
 AI : 빈도분석은 토큰화된 텍스트에서 각 단어의 출현 빈도를 계산하여 텍스트의 주요 주제나 핵심 키워드를 파악하는 과정입니다. 이를 통해 텍스트 데이터의 중요한 내용을 이해할 수 있습니다. (참조: './pdf_data//03_02_텍스트데이터분석1_빈도분석_V10.pdf' - page 0)
USER :(quit or q : 종료)  q
